# ML.NET Image Classification APIを使用した画像分類（転移学習）

ML.NET Image Classification API を使用することで画像分類ができる。事前トレーニング済みの`ResNet v2`を使用して転移学習する。

In [6]:
#r "nuget:Microsoft.ML.Vision, 1.4.0"
#r "nuget:SciSharp.TensorFlow.Redist , 1.15.0"
#r "nuget:Microsoft.ML.ImageAnalytics, 1.4.0"

Installing package Microsoft.ML.Vision, version 1.4.0.done!

Successfully added reference to package Microsoft.ML.Vision, version

Installing package SciSharp.TensorFlow.Redist, version 1.15.0.done!

Successfully added reference to package SciSharp.TensorFlow.Redist, version

Installing package Microsoft.ML.ImageAnalytics, version 1.4.0.done!

Successfully added reference to package Microsoft.ML.ImageAnalytics, version

In [7]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using Microsoft.ML;
using static Microsoft.ML.DataOperationsCatalog;
using Microsoft.ML.Vision;

In [8]:
public class ImageData
{
    public string ImagePath { get; set; }
    public string Label { get; set; }
}

## 入出力データのスキーマ

In [9]:
// 入力データスキーマ
public class ModelInput
{
    // 画像のバイト表現
    public byte[] Image { get; set; }
    // ラベルの数値表現
    public UInt32 LabelAsKey { get; set; }
    // 画像の完全修飾パス
    public string ImagePath { get; set; }
    // ラベル（予測すべき値）
    public string Label { get; set; }
}

// 出力データスキーマ
public class ModelOutput
{
    // 画像の完全修飾パス
    public string ImagePath { get; set; }
    // 正解ラベル
    public string Label { get; set; }
    // 予測ラベル
    public string PredictedLabel { get; set; }
}

## パスの定義と変数の初期化

In [10]:
var projectDirectory = Path.GetFullPath(Environment.CurrentDirectory);
var workspaceRelativePath = Path.Combine(projectDirectory, "workspace");
var assetsRelativePath = Path.Combine(projectDirectory, "data", "concrete_deck_cracked");

display(projectDirectory);
display(workspaceRelativePath);
display(assetsRelativePath);

C:\Data\Study\Jupyter\jupyter_notes\notes_dotnet\MLNet

C:\Data\Study\Jupyter\jupyter_notes\notes_dotnet\MLNet\workspace

C:\Data\Study\Jupyter\jupyter_notes\notes_dotnet\MLNet\data\concrete_deck_cracked

In [11]:
MLContext mlContext = new MLContext();
var useFolderNameAsLabel = true;

In [12]:
// 指定したディレクトリの画像をImageDataのコレクションとして取得するメソッド
public static IEnumerable<ImageData> LoadImagesFromDirectory(string folder, bool useFolderNameAsLabel = true)
{
    // folder内のすべてのファイル
    var files = Directory.GetFiles(folder, "*", searchOption: SearchOption.AllDirectories);

    foreach (var file in files)
    {
        // ファイル拡張子がAPIでサポートされているjpg/pngであることを確認
        if ((Path.GetExtension(file) != ".jpg") && (Path.GetExtension(file) != ".png")) continue;
    
        var label = Path.GetFileName(file);

        if (useFolderNameAsLabel) 
        {
            label = Directory.GetParent(file).Name;
        }
        else
        {
            for (int index = 0; index < label.Length; index++)
            {
                if (!char.IsLetter(label[index]))
                {
                    label = label.Substring(0, index);
                    break;
                }
            }
        }
    
        yield return new ImageData(){ImagePath = file, Label = label};
    }
}

## データの読み込み

In [36]:
IEnumerable<ImageData> images = LoadImagesFromDirectory(folder: assetsRelativePath, useFolderNameAsLabel: true);
display(images.Take(5));

index,ImagePath,Label
0,C:\Data\Study\Jupyter\jupyter_notes\notes_dotnet\MLNet\data\concrete_deck_cracked\CD\7001-115.jpg,CD
1,C:\Data\Study\Jupyter\jupyter_notes\notes_dotnet\MLNet\data\concrete_deck_cracked\CD\7001-139.jpg,CD
2,C:\Data\Study\Jupyter\jupyter_notes\notes_dotnet\MLNet\data\concrete_deck_cracked\CD\7001-151.jpg,CD
3,C:\Data\Study\Jupyter\jupyter_notes\notes_dotnet\MLNet\data\concrete_deck_cracked\CD\7001-157.jpg,CD
4,C:\Data\Study\Jupyter\jupyter_notes\notes_dotnet\MLNet\data\concrete_deck_cracked\CD\7001-169.jpg,CD


`IDataView`にデータをロードする。`IDataView`はLINQでいうところの`IEnumerable<T>`に相当する、ML.NETのデータの中心となる存在。 [参考](https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.idataview?view=ml-dotnet)

In [14]:
// 読み込んだImageDataをIDataViewにロード
IDataView imageData = mlContext.Data.LoadFromEnumerable(images);
// シャッフル
IDataView shuffledData = mlContext.Data.ShuffleRows(imageData);

In [15]:
// ラベルと画像を数値に変換する EstimatorChain
var preprocessingPipeline = 
    mlContext.Transforms.Conversion
    .MapValueToKey(
        inputColumnName: "Label",
        outputColumnName: "LabelAsKey")
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: assetsRelativePath,
        inputColumnName: "ImagePath"));

In [16]:
// 前処理
IDataView preProcessedData = preprocessingPipeline
                    .Fit(shuffledData)
                    .Transform(shuffledData);

In [17]:
// 訓練データとテストデータに分割
TrainTestData trainSplit = mlContext.Data.TrainTestSplit(data: preProcessedData, testFraction: 0.3);
TrainTestData validationTestSplit = mlContext.Data.TrainTestSplit(trainSplit.TestSet);

IDataView trainSet = trainSplit.TrainSet;
IDataView validationSet = validationTestSplit.TrainSet;
IDataView testSet = validationTestSplit.TestSet;

## トレーニングパイプライン

In [29]:
var classifierOptions = new ImageClassificationTrainer.Options()
{
    // 入力列
    FeatureColumnName = "Image",
    // ラベル
    LabelColumnName = "LabelAsKey",
    // 検量データセット
    ValidationSet = validationSet,
    // モデルアーキテクチャ（Resnet v2 を指定）
    Arch = ImageClassificationTrainer.Architecture.ResnetV2101,
    // トレーニング中の進捗記録用コールバック
    MetricsCallback = (metrics) => Console.WriteLine(metrics),
    // 検証セットがない時に訓練セットで検証
    TestOnTrainSet = false,
    // 後続の実行でボトルネックフェーズからキャッシュされた値を使うか
    ReuseTrainSetBottleneckCachedValues = true,
    ReuseValidationSetBottleneckCachedValues = true,
    // 計算されたボトルネック値を格納するティレクトリ
    WorkspacePath=workspaceRelativePath
};

In [30]:
var trainingPipeline = mlContext.MulticlassClassification.Trainers
    .ImageClassification(classifierOptions)
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

In [31]:
ITransformer trainedModel = trainingPipeline.Fit(trainSet);

Saver not created because there are no variables in the graph to restore
Phase: Training, Dataset used: Validation, Batch Processed Count:  12, Epoch:   0, Accuracy:  0.6916666
Phase: Training, Dataset used: Validation, Batch Processed Count:  12, Epoch:   1, Accuracy: 0.71250004
Phase: Training, Dataset used: Validation, Batch Processed Count:  12, Epoch:   2, Accuracy:  0.7541666
Phase: Training, Dataset used: Validation, Batch Processed Count:  12, Epoch:   3, Accuracy: 0.53333336
Phase: Training, Dataset used: Validation, Batch Processed Count:  12, Epoch:   4, Accuracy: 0.70416665
Phase: Training, Dataset used: Validation, Batch Processed Count:  12, Epoch:   5, Accuracy: 0.62500006
Phase: Training, Dataset used: Validation, Batch Processed Count:  12, Epoch:   6, Accuracy:  0.8083334
Phase: Training, Dataset used: Validation, Batch Processed Count:  12, Epoch:   7, Accuracy: 0.56666666
Phase: Training, Dataset used: Validation, Batch Processed Count:  12, Epoch:   8, Accuracy: 0.

## 予測

In [32]:
private static void OutputPrediction(ModelOutput prediction)
{
    string imageName = Path.GetFileName(prediction.ImagePath);
    Console.WriteLine($"Image: {imageName} | Actual Value: {prediction.Label} | Predicted Value: {prediction.PredictedLabel}");
}

public static void ClassifySingleImage(MLContext mlContext, IDataView data, ITransformer trainedModel)
{
    PredictionEngine<ModelInput, ModelOutput> predictionEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(trainedModel);
    ModelInput image = mlContext.Data.CreateEnumerable<ModelInput>(data,reuseRowObject:true).First();
    
    // Prediction
    ModelOutput prediction = predictionEngine.Predict(image);
    
    Console.WriteLine("Classifying single image");
    OutputPrediction(prediction);
}

In [33]:
ClassifySingleImage(mlContext, testSet, trainedModel);

Classifying single image
Image: 7001-120.jpg | Actual Value: UD | Predicted Value: CD


## 複数枚予測

In [34]:
public static void ClassifyImages(MLContext mlContext, IDataView data, ITransformer trainedModel)
{
    IDataView predictionData = trainedModel.Transform(data);
    IEnumerable<ModelOutput> predictions = mlContext.Data.CreateEnumerable<ModelOutput>(predictionData, reuseRowObject: true).Take(10);
    Console.WriteLine("Classifying multiple images");
    foreach (var prediction in predictions)
    {
        OutputPrediction(prediction);
    }
}

In [35]:
ClassifyImages(mlContext, testSet, trainedModel);

Classifying multiple images
Image: 7001-120.jpg | Actual Value: UD | Predicted Value: CD
Image: 7005-80.jpg | Actual Value: CD | Predicted Value: UD
Image: 7002-194.jpg | Actual Value: CD | Predicted Value: UD
Image: 7001-228.jpg | Actual Value: UD | Predicted Value: CD
Image: 7001-211.jpg | Actual Value: UD | Predicted Value: CD
Image: 7001-42.jpg | Actual Value: CD | Predicted Value: UD
